In [ ]:
import sounddevice as sd
import soundfile as sf
import queue
import threading
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
from scipy.fftpack import fft, ifft, fftfreq

dB = lambda x: 20*np.log10(np.abs(x))

In [ ]:
devices = sd.query_devices()
for device in devices:
    if device['name'] == 'UltraLite-mk4':
        sd.default.device = device['name']
        sd.default.samplerate = device['default_samplerate']
        fs = int(device['default_samplerate'])
    

In [ ]:
T = 5
n=fs*T
t=np.arange(n)/fs
f=fftfreq(n,1/fs)
fMin = 1
fMax = 90e3#f.max()
chirp = signal.waveforms.chirp(t,fMin,t[-1],fMax,method='log',phi=-90)

In [ ]:
q = queue.Queue()
event = threading.Event()

filename = 'rostock_lasVegasTango_short.wav'
blocksize = 2048
buffers = 24

def callback(outdata, frames, time, status):
    assert frames == blocksize
    if status.output_underflow:      
        print('Output underflow: increase blocksize?', file=sys.stderr)
        raise sd.CallbackAbort
    assert not status
    try:
        data = q.get_nowait()
    except queue.Empty:
        print('Buffer is empty: increase buffers?', file=sys.stderr)
        raise sd.CallbackAbort
    if len(data) < len(outdata):
        outdata[:len(data)] = data
        outdata[len(data):] = np.zeros((len(outdata) - len(data),1))
        raise sd.CallbackStop
    else:
        outdata[:] = data

with sf.SoundFile(filename) as fi:
    channels = fi.channels
    for _ in range(buffers):
        data = fi.buffer_read(blocksize, dtype='float32')
        if not data:
            break
        datanp = np.frombuffer(data,dtype='float32')
        datanp.shape = -1, channels
        q.put_nowait(datanp)  # Pre-fill queue        
    stream = sd.OutputStream(
        samplerate=fi.samplerate, blocksize=blocksize, 
        device='UltraLite-mk4', channels=fi.channels, dtype='float32', 
        callback=callback, finished_callback=event.set)
    with stream:
        timeout = blocksize * buffers / fi.samplerate
        while data:
            data = fi.buffer_read(blocksize, dtype='float32')
#            print(len(data))
            datanp = np.frombuffer(data,dtype='float32')
            datanp.shape = -1, channels
#            print(datanp.shape)
            q.put(datanp, timeout=timeout)
        event.wait()  # Wait until playback is finished

In [ ]:
inQ = queue.Queue()
outQ = queue.Queue()
event = threading.Event()
blocksize = 2048
buffers = 24

sOut = chirp.astype('float32')
totalBlocks = len(sOut) // blocksize
        
def callback(outdata, frames, time, status):
    data = outQ.get_nowait()
    if len(data) < len(outdata):
        outdata[:len(data)] = data
        outdata[len(data):] = np.zeros((len(outdata) - len(data),1))
        raise sd.CallbackStop
    else:
        outdata[:] = data
    
for outBlock in range(buffers):
    data = sOut[outBlock*blocksize:(outBlock+1)*blocksize]
    data.shape = -1,1
    outQ.put(data)
    
stream = sd.OutputStream(
    samplerate=fs, blocksize=blocksize, 
    device = 'UltraLite-mk4', channels=1, dtype='float32',
    callback=callback, finished_callback=event.set)
print(stream.active)
with stream: 
    timeout = blocksize*buffers/fs
    for outBlock in range(buffers,totalBlocks):
        data = sOut[outBlock*blocksize:(outBlock+1)*blocksize]
        data.shape = -1 ,1
        outQ.put(data, timeout=timeout)
    event.wait()

In [ ]:
sOut = chirp
channels = sOut.size//n
sIn = sd.playrec(sOut, blocking = True,channels=channels)
#sIn = np.reshape(sIn,sOut.shape)

In [ ]:
win = signal.windows.hann(n)
win = signal.windows.boxcar(n)
SOut = fft(sOut*win,axis=0)/win.sum()
SIn = fft(sIn*win,axis=0)/win.sum()
fIdx = (f>fMin) & (f<fMax)
#fIdx = (f>30e3) & (f<90e3)
dBrange = 20

H = SIn/SOut
H /= np.max(np.abs(H))
plt.semilogx(f[fIdx],dB(H[fIdx]))
plt.ylim(ymin=-dBrange, ymax=0.5)
plt.show()